https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb From this link with different data

In [3]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [4]:
from tensorflow import keras

In [5]:
import tensorflow as tf
# from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Embedding
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [6]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from sklearn.metrics import classification_report

In [7]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLDER = "../../../"
%cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

sys.path.insert(0, 'Models/PreProcessing/')

from char_utils import *

In [ ]:
DAMAN_CSV = "Models/Data/ERData/daman.csv"

er_df = ERData_MajorityLabel()
# e_df = ERStateData_with_freq(DAMAN_CSV)

In [ ]:
# er_df = pd.DataFrame()
# er_df['Name'] = e_df['Name']
# er_df['Gender'] = e_df['new_labels']

In [ ]:
er_df.groupby('Gender')['Gender'].value_counts()

In [ ]:
train_x, train_y, test_x, test_y = splitERData(er_df, 'Name','Models/ERData/'+ 'char_lstm_er_oov.txt')

In [ ]:
voc = vocab()

In [43]:
test_x.shape[0]+train_x.shape[0]

6353946

#### https://stackoverflow.com/questions/59265920/this-tensorflow-binary-is-optimized-with-intelr-mkl-dnn-to-use-the-following-c

https://stackoverflow.com/questions/53698035/failed-to-get-convolution-algorithm-this-is-probably-because-cudnn-failed-to-in 

In [ ]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [28]:
MAXLEN_NAME_CHAR_CNN = 30
inputs = Input(MAXLEN_NAME_CHAR_CNN)
x = Embedding(len(voc), len(voc))(inputs)
x = Conv1D(filters = 256, kernel_size = 7, activation = 'relu')(x)
x = Flatten()(x)
predictions = Dense(2, activation='softmax')(x)
cnn_model = Model(inputs=inputs, outputs=predictions)
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [29]:
cnn_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 30, 28)            784       
                                                                 
 conv1d_2 (Conv1D)           (None, 24, 256)           50432     
                                                                 
 flatten_2 (Flatten)         (None, 6144)              0         
                                                                 
 dense_2 (Dense)             (None, 2)                 12290     
                                                                 
Total params: 63,506
Trainable params: 63,506
Non-trainable params: 0
_________________________________________________________________


In [36]:
batch_size=1000
cnn_model.fit(train_x, train_y,batch_size=batch_size,epochs=15,validation_data=(test_x, test_y))

Epoch 1/15
4448/4448 [==============================] - 285s 64ms/step - loss: 0.2229 - accuracy: 0.9104 - val_loss: 0.2331 - val_accuracy: 0.9063
Epoch 2/15
4448/4448 [==============================] - 270s 61ms/step - loss: 0.2223 - accuracy: 0.9106 - val_loss: 0.2331 - val_accuracy: 0.9063
Epoch 3/15
4448/4448 [==============================] - 274s 62ms/step - loss: 0.2217 - accuracy: 0.9110 - val_loss: 0.2326 - val_accuracy: 0.9065
Epoch 4/15
4448/4448 [==============================] - 275s 62ms/step - loss: 0.2212 - accuracy: 0.9112 - val_loss: 0.2328 - val_accuracy: 0.9066
Epoch 5/15
4448/4448 [==============================] - 277s 62ms/step - loss: 0.2209 - accuracy: 0.9113 - val_loss: 0.2328 - val_accuracy: 0.9065
Epoch 6/15
4448/4448 [==============================] - 274s 62ms/step - loss: 0.2207 - accuracy: 0.9114 - val_loss: 0.2331 - val_accuracy: 0.9067
Epoch 7/15
4448/4448 [==============================] - 273s 61ms/step - loss: 0.2204 - accuracy: 0.9116 - val_loss: 0

In [ ]:
# import keras
# import tensorflow as tf
# import tensorflow.keras.backend as K


# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 0.6 sometimes works better for folks
# tf.keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [37]:
score, acc = cnn_model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

59568/59568 [==============================] - 133s 2ms/step - loss: 0.2322 - accuracy: 0.9068
Test score: 0.2321975827217102
Test accuracy: 0.9068312644958496


In [38]:
pred = cnn_model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male     0.9010    0.9259    0.9133   1010138
      Female     0.9138    0.8853    0.8993    896016

    accuracy                         0.9068   1906154
   macro avg     0.9074    0.9056    0.9063   1906154
weighted avg     0.9070    0.9068    0.9067   1906154

--------------------------------------------------------


In [39]:
filename = "Models/ERData/CharCNN_CharLSTM/SavedModel/CharCNN/"
cnn_model.save(filename)

INFO:tensorflow:Assets written to: Models/ERData/CharCNN_CharLSTM/SavedModel/CharCNN/assets


In [40]:
%pwd

'/home/souvic/mounted/btp/vahini/Name2Demographics'

In [46]:
inputs = Input(MAXLEN_NAME_CHAR_LSTM)
x = Embedding(len(voc), len(voc))(inputs)
x = LSTM(512, return_sequences=True, activation='tanh')(x)
x = Dropout(0.2)(x)
x = LSTM(512, return_sequences=False, activation='tanh')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
batch_size=1000
model.fit(train_x, train_y,batch_size=batch_size,epochs=15,validation_data=(test_x, test_y))

Epoch 1/15
4448/4448 [==============================] - 14401s 3s/step - loss: 0.3582 - accuracy: 0.8354 - val_loss: 0.2503 - val_accuracy: 0.8949
Epoch 2/15
4448/4448 [==============================] - 13982s 3s/step - loss: 0.2237 - accuracy: 0.9078 - val_loss: 0.2073 - val_accuracy: 0.9156
Epoch 3/15
1602/4448 [=========>....................] - ETA: 2:10:20 - loss: 0.1979 - accuracy: 0.9199

In [ ]:
score, acc = model.evaluate(test_x, test_y)

In [60]:
print(1)

1


In [57]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.2195446640253067
Test accuracy: 0.92354816198349


In [58]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male     0.9262    0.9298    0.9280   1010138
      Female     0.9205    0.9165    0.9185    896016

    accuracy                         0.9235   1906154
   macro avg     0.9234    0.9231    0.9233   1906154
weighted avg     0.9235    0.9235    0.9235   1906154

--------------------------------------------------------


In [59]:
report = classification_report(y_true, y_pred, target_names=target_names, digits=4)